In [ ]:
# Преоразовать существующий код для распознавания последовательности рукописных символов в текст.
# Написать генератор произвольной последовательности рукописных символов.

# ГЕНЕРАЦИЯ НАБОРА ДАННЫХ
# Сгенерировать набор текстовых строк из допустимых символов.
# Каждому набору поставить в соотвествие массив изоражений соответствующих символов.
# Добавить пробелы для разделения слов, символы пробелов не использовать во время классификации.

# КЛАССИФИКАЦИЯ (k ближайших соседей)
# Часть символов оставить для обучения классификатора . Остальные  классифицировать
# по аналогии с примером mnist с прошлого занятия. Т.е. преобразовывать изображение в вектор.
# Выбрать оптимальный классификатор в зависимости от k- количество соседей.
# Метрика для измерения точности классификации, и выбора оптимального классификатора относительно параметра k - accuracy.

# РАСПОЗНАВАНИЕ
# Итак, к данному этапу Вы из массива изображений "восстановили" текст.
# Случайно удалите некоторые символы,и случайно добавьте символы в результат распознавания.
# Рассчитайте метрику CER.


In [ ]:
import os
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import Levenshtein
%matplotlib inline

In [ ]:
COUNT_NUMBERS_IN_STRINGS = 10
COUNT_STRINGS = 10
IMAGE_SIZE = (28, 28)

In [ ]:
#Функции для загрузки и отображения данных
def load_images(directory):
    """
    Загрузка изображений из директории, где каждая подпапка содержит
    множество изображений одного символа
    """
    images_dict = {}
    # Получаем список папок, заменяем 'times' на '*' только для отображения
    raw_symbols = [symb for symb in os.listdir(directory)]
    symbols = [s.replace('times', '*') for s in raw_symbols]

    for raw_symbol, display_symbol in zip(raw_symbols, symbols):
        symbol_path = os.path.join(directory, raw_symbol)
        images_dict[display_symbol] = []

        # Загрузка всех изображений для текущего символа
        for img_file in os.listdir(symbol_path):
            if img_file.endswith('.jpg'):
                img_path = os.path.join(symbol_path, img_file)
                img = Image.open(img_path).convert('L')
                img = img.resize(IMAGE_SIZE)
                images_dict[display_symbol].append(np.array(img).flatten())

    return images_dict, symbols

def display_sample_images(directory, symbols, num_symbols=3, samples_per_symbol=4):
    """Отображение примеров изображений для каждого символа"""
    sample_symbols = random.sample(symbols, min(num_symbols, len(symbols)))

    fig, axes = plt.subplots(len(sample_symbols), samples_per_symbol,
                            figsize=(15, 3*len(sample_symbols)))
    if len(sample_symbols) == 1:
        axes = [axes]

    for i, symbol in enumerate(sample_symbols):
        # Преобразование обратно для чтения файлов
        folder_symbol = symbol.replace('*', 'times')
        symbol_path = os.path.join(directory, folder_symbol)
        image_files = [f for f in os.listdir(symbol_path) if f.endswith('.jpg')]
        sample_files = random.sample(image_files,
                                   min(samples_per_symbol, len(image_files)))

        for j, img_file in enumerate(sample_files):
            img_path = os.path.join(symbol_path, img_file)
            img = Image.open(img_path).convert('L')

            if len(sample_symbols) > 1:
                ax = axes[i][j]
            else:
                ax = axes[j]

            ax.imshow(img, cmap='gray')
            ax.axis('off')
            if j == 0:
                ax.set_title(f'Symbol: {symbol}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Загрузка данных и отображение примеров
directory = 'data/data'
images_dict, symbols = load_images(directory)

print(f'Доступные символы: {symbols}')
print("\nКоличество изображений для каждого символа:")
for symbol, images in images_dict.items():
    print(f"{symbol}: {len(images)} изображений")

display_sample_images(directory, symbols)


In [ ]:
# Функции для генерации датасета
def generate_dataset(symbols, count_strings, count_symbols):
    """Генерация набора данных"""
    strings = [''.join(random.choices(symbols, k=count_symbols))
              for _ in range(count_strings)]
    strings_with_spaces = [' '.join(list(s)) for s in strings]
    return strings, strings_with_spaces

def prepare_data(images_dict, strings):
    """Подготовка данных для классификации"""
    X = []
    y = []

    for string in strings:
        for char in string:
            if char in images_dict:
                # Случайно выбираем одно изображение для символа
                random_image = random.choice(images_dict[char])
                X.append(random_image)
                y.append(char)

    return np.array(X), np.array(y)

# Генерация данных
strings, strings_with_spaces = generate_dataset(symbols,
                                              COUNT_STRINGS,
                                              COUNT_NUMBERS_IN_STRINGS)

print("Примеры сгенерированных строк:")
for i, (s, s_space) in enumerate(zip(strings[:5], strings_with_spaces[:5])):
    print(f"{i+1}. Без пробелов: {s}")
    print(f"   С пробелами:  {s_space}\n")


In [ ]:
# Подготовка расширенного набора данных для классификации
def prepare_extended_data(images_dict, strings):
    """Подготовка расширенного набора данных, используя все доступные изображения"""
    X = []
    y = []

    for string in strings:
        for char in string:
            if char in images_dict:
                # Добавляем все изображения для каждого символа
                X.extend(images_dict[char])
                y.extend([char] * len(images_dict[char]))

    return np.array(X), np.array(y)

X, y = prepare_extended_data(images_dict, strings)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")


In [ ]:
# Поиск оптимального k
def find_optimal_k(X_train, X_test, y_train, y_test, k_range):
    """Поиск оптимального k для KNN"""
    accuracies = []
    for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))

    best_k = k_range[np.argmax(accuracies)]
    return best_k, accuracies

k_range = range(1, 11)
best_k, accuracies = find_optimal_k(X_train, X_test, y_train, y_test, k_range)

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.plot(k_range, accuracies, 'bo-')
plt.xlabel('Количество соседей (k)')
plt.ylabel('Точность')
plt.title('Зависимость точности от количества соседей')
plt.grid(True)
plt.show()

print(f"Оптимальное значение k: {best_k}")
print(f"Лучшая точность: {max(accuracies):.4f}")


In [ ]:
# Симуляция распознавания и расчет CER
def calculate_cer(original, predicted):
    """Расчет Character Error Rate"""
    distance = Levenshtein.distance(original, predicted)
    return distance / len(original)

# Финальная модель с оптимальным k
final_knn = KNeighborsClassifier(n_neighbors=best_k)
final_knn.fit(X_train, y_train)

# Симуляция распознавания с использованием случайных изображений
recognized_strings = []
for string in strings:
    chars = list(string)
    # Симуляция ошибок распознавания
    if len(chars) > 2:
        # Удаление случайного символа
        del_pos = random.randint(0, len(chars)-1)
        chars.pop(del_pos)

    # Добавление случайного символа
    add_pos = random.randint(0, len(chars))
    chars.insert(add_pos, random.choice(symbols))

    recognized_strings.append(''.join(chars))

# Расчет CER
cer_scores = [calculate_cer(original, recognized)
             for original, recognized in zip(strings, recognized_strings)]

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.hist(cer_scores, bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Character Error Rate (CER)')
plt.ylabel('Частота')
plt.title('Распределение CER')
plt.grid(True)
plt.show()

print("\nПримеры распознавания:")
for orig, rec, cer in zip(strings[:5], recognized_strings[:5], cer_scores[:5]):
    print(f"Оригинал:   {orig}")
    print(f"Распознано: {rec}")
    print(f"CER:        {cer:.4f}\n")

print(f"Средний CER: {np.mean(cer_scores):.4f}")
